In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

file_path = 'pcanewss.xlsx'
pca_data = pd.read_excel(file_path)

pca_data.head()

,logreturn,f1,f2,f3,f4,f5,f6,f7
0,0.014110,4.798603,4.291953,0.096115,-0.915052,1.665008,0.334631,-0.286816
1,0.004389,4.777986,4.650318,0.538249,-0.841286,1.652642,0.600573,0.067374
2,-0.009372,4.764654,4.265499,0.066010,-1.018443,1.649977,0.753251,-0.054899
3,0.005736,4.723674,4.242907,0.049920,-1.107658,1.669910,0.458335,-0.511413
4,0.011969,4.720298,4.265986,0.088056,-0.870258,1.729706,0.997562,0.209364


In [2]:
X = pca_data[['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7']]
y = pca_data['logreturn']

In [3]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

val_size = int(len(X_train) * 0.1)
X_train, X_val = X_train[:-val_size], X_train[-val_size:]
y_train, y_val = y_train[:-val_size], y_train[-val_size:]

In [4]:
X_train = X_train.values 
X_val = X_val.values
X_test = X_test.values

y_train = y_train.values.reshape(-1, 1) 
y_val = y_val.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

scaler_x = StandardScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
X_val_scaled = scaler_x.transform(X_val)
X_test_scaled = scaler_x.transform(X_test)

In [5]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [6]:
class BPNeuralNetwork(nn.Module):
    def __init__(self):
        super(BPNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(7, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
model = BPNeuralNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-4)

In [8]:
patience = 20000 #
best_val_loss = float('inf')
counter = 0

num_epochs = 1200

Due to the instability of the model, we previously applied early stopping with a patience of 10 or 20. However, we ultimately decided to increase the patience and experimented with different learning rates and numbers of epochs. We found that under our chosen learning rate, the model generally converges when the number of epochs reaches 1200.

In [9]:
for epoch in range(num_epochs):
    # Training
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.flatten(), y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs.flatten(), y_val_tensor)

    # Early Stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Val Loss: {val_loss.item()}")


/opt/anaconda3/envs/work/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1924, 1])) that is different to the input size (torch.Size([1924])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/envs/work/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([213, 1])) that is different to the input size (torch.Size([213])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/1200], Loss: 0.005560179706662893, Val Loss: 0.04191077873110771
Epoch [20/1200], Loss: 0.0017148987390100956, Val Loss: 0.0025222147814929485
Epoch [30/1200], Loss: 0.000407375511713326, Val Loss: 0.008438694290816784
Epoch [40/1200], Loss: 0.000324101943988353, Val Loss: 0.005788078065961599
Epoch [50/1200], Loss: 0.0002788090496324003, Val Loss: 0.0034711314365267754
Epoch [60/1200], Loss: 0.00020498175581451505, Val Loss: 0.004968451801687479
Epoch [70/1200], Loss: 0.00016823105397634208, Val Loss: 0.003554401220753789
Epoch [80/1200], Loss: 0.00015059334691613913, Val Loss: 0.0034991793800145388
Epoch [90/1200], Loss: 0.00013892431161366403, Val Loss: 0.003002696903422475
Epoch [100/1200], Loss: 0.00013020384358242154, Val Loss: 0.0025182405952364206
Epoch [110/1200], Loss: 0.00012338411761447787, Val Loss: 0.002128029242157936
Epoch [120/1200], Loss: 0.00011793355224654078, Val Loss: 0.0017689617816358805
Epoch [130/1200], Loss: 0.00011332848953315988, Val Loss: 0.00160

Epoch [1060/1200], Loss: 8.008881559362635e-05, Val Loss: 0.00014971510972827673
Epoch [1070/1200], Loss: 8.00832494860515e-05, Val Loss: 0.0001492217561462894
Epoch [1080/1200], Loss: 8.007803262444213e-05, Val Loss: 0.00014872648171149194
Epoch [1090/1200], Loss: 8.007309952517971e-05, Val Loss: 0.0001482562511228025
Epoch [1100/1200], Loss: 8.00684210844338e-05, Val Loss: 0.00014780017954763025
Epoch [1110/1200], Loss: 8.006417192518711e-05, Val Loss: 0.00014736410230398178
Epoch [1120/1200], Loss: 8.006020652828738e-05, Val Loss: 0.00014694598212372512
Epoch [1130/1200], Loss: 8.005636482266709e-05, Val Loss: 0.00014655340055469424
Epoch [1140/1200], Loss: 8.005272684386e-05, Val Loss: 0.0001462092186557129
Epoch [1150/1200], Loss: 8.004916890058666e-05, Val Loss: 0.0001459022460039705
Epoch [1160/1200], Loss: 8.004571282071993e-05, Val Loss: 0.00014561980788130313
Epoch [1170/1200], Loss: 8.004235860425979e-05, Val Loss: 0.00014535561786033213
Epoch [1180/1200], Loss: 8.0039077147

In [10]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor).flatten() 
    test_loss = criterion(test_outputs, y_test_tensor.flatten()) 

    y_pred = test_outputs.numpy()
    y_test_array = y_test_tensor.numpy()

    test_mse = mean_squared_error(y_test_array, y_pred)

    print(f"Test MSE (original scale): {test_mse}")

correlation = np.corrcoef(y_test_array.flatten(), y_pred.flatten())[0, 1]
print(f"Correlation between y_test and y_pred: {correlation}")

Test MSE (original scale): 6.213107553776354e-05
Correlation between y_test and y_pred: 0.03058755727879976


# Correlation

In [18]:
model = BPNeuralNetwork()

def negative_correlation_loss(output, target):

    output = output - torch.mean(output)  
    target = target - torch.mean(target)  
    numerator = torch.sum(output * target) 
    denominator = torch.sqrt(torch.sum(output ** 2)) * torch.sqrt(torch.sum(target ** 2))  
    correlation = numerator / (denominator + 1e-8)  
    return -correlation 

criterion = negative_correlation_loss
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

patience = 10000  # Early stopping patience
best_val_loss = float('inf')
counter = 0
num_epochs = 60

for epoch in range(num_epochs):
    # Training
    model.train()
    outputs = model(X_train_tensor).flatten()
    loss = criterion(outputs, y_train_tensor.flatten())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor).flatten()
        val_loss = criterion(val_outputs, y_val_tensor.flatten())

    # Calculate correlation for training
    train_correlation = np.corrcoef(
        y_train_tensor.numpy().flatten(), outputs.detach().numpy().flatten()
    )[0, 1]

    # Calculate correlation for validation
    val_correlation = np.corrcoef(
        y_val_tensor.numpy().flatten(), val_outputs.numpy().flatten()
    )[0, 1]

    # Early Stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    # Print metrics every 10 epochs or at the last epoch
    if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        print(
            f"Epoch [{epoch+1}/{num_epochs}], "
            f"Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
            f"Train Correlation: {train_correlation:.4f}, Val Correlation: {val_correlation:.4f}"
        )

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor).flatten()
    test_loss = criterion(test_outputs, y_test_tensor.flatten()).item()

    y_pred = test_outputs.numpy()
    y_test_array = y_test_tensor.numpy()

    test_correlation = np.corrcoef(y_test_array.flatten(), y_pred.flatten())[0, 1]

    print(f"Test Loss (Negative Correlation): {test_loss}")
    print(f"Correlation between y_test and y_pred: {test_correlation}")

Epoch [10/60], Loss: -0.0484, Val Loss: -0.0710, Train Correlation: 0.0484, Val Correlation: 0.0710
Epoch [20/60], Loss: -0.0546, Val Loss: -0.0728, Train Correlation: 0.0546, Val Correlation: 0.0728
Epoch [30/60], Loss: -0.0608, Val Loss: -0.0745, Train Correlation: 0.0608, Val Correlation: 0.0745
Epoch [40/60], Loss: -0.0672, Val Loss: -0.0763, Train Correlation: 0.0672, Val Correlation: 0.0763
Epoch [50/60], Loss: -0.0737, Val Loss: -0.0780, Train Correlation: 0.0737, Val Correlation: 0.0780
Epoch [60/60], Loss: -0.0800, Val Loss: -0.0793, Train Correlation: 0.0800, Val Correlation: 0.0793
Test Loss (Negative Correlation): 0.009302489459514618
Correlation between y_test and y_pred: -0.009302484298110007


# Linear Regression

In [19]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression()

lr.fit(X_train_scaled, y_train)

y_train_pred = lr.predict(X_train_scaled)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
print(f"Training MSE: {train_mse}")

y_val_pred = lr.predict(X_val_scaled)
val_mse = mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)
print(f"Validation MSE: {val_mse}")

y_test_pred = lr.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test MSE: {test_mse}")

correlation = np.corrcoef(y_test.flatten(), y_test_pred.flatten())[0, 1]
print(f"Correlation between y_test and y_test_pred: {correlation}")

Training MSE: 7.965149397262919e-05
Validation MSE: 0.00014415698439906142
Test MSE: 6.294942001220667e-05
Correlation between y_test and y_test_pred: 0.04226254917424558


# Test

In [20]:
y_test_pred.shape

(535, 1)

In [21]:
y_test_naive = np.array([0] * 535).reshape(535, 1)
y_test_naive.shape

(535, 1)

In [22]:
mean_squared_error(y_test, y_test_naive)

6.158865164661923e-05

# Original Dataset

In [50]:
new_file_path = 'projectdata.xlsx'
data = pd.read_excel(new_file_path)

In [51]:
data.head(5)

,date,price,logreturn,HS300,CompriceIndex,PE,S&P500Volatility3,deprate,deprate3,deprate6,...,setpriceCOMEXsilver,Londonspotgold,Londonspotsilver,SHgoldspotvolumeAu9999,SHgoldspotClspriceAu9999,Goldproduction,Goldcondemand,USGoldcondemand,tradevolume,position
0,2012-01-05,331.95,0.014110,2276.385,0.00,13.9824,24.70,0.5,3.1,3.3,...,29.296,1032.546,28.92,7869.6,331.98,19.733,259.79,14.05,82316,99406
1,2012-01-06,333.41,0.004389,2290.601,155.28,13.9454,24.09,0.5,3.1,3.3,...,28.683,1049.880,29.40,9875.8,332.58,19.733,259.79,14.05,95518,91782
2,2012-01-09,330.30,-0.009372,2368.570,0.00,13.9753,24.12,0.5,3.1,3.3,...,28.782,1045.443,28.85,9369.4,331.04,19.733,259.79,14.05,69482,90598
3,2012-01-10,332.20,0.005736,2447.349,0.00,14.1068,23.68,0.5,3.1,3.3,...,29.815,1056.947,29.69,8523.4,332.05,19.733,259.79,14.05,63840,86898
4,2012-01-11,336.20,0.011969,2435.608,0.00,14.1205,24.11,0.5,3.1,3.3,...,29.890,1064.405,29.81,5768.8,335.50,19.733,259.79,14.05,61830,85756


In [52]:
data['price'].head(5)

0    331.95
1    333.41
2    330.30
3    332.20
4    336.20
Name: price, dtype: float64

In [53]:
original_y = data['logreturn']
original_X = data.drop(columns=['date', 'price', 'logreturn']) 

In [54]:
original_X.shape

(2672, 29)

In [55]:
original_train_size = int(len(original_X) * 0.8)
original_X_train, original_X_test = X[:original_train_size], X[original_train_size:]
original_y_train, original_y_test = y[:original_train_size], y[original_train_size:]

val_size = int(len(original_X_train) * 0.1)
original_X_train, original_X_val = original_X_train[:-val_size], original_X_train[-val_size:]
original_y_train, original_y_val = original_y_train[:-val_size], original_y_train[-val_size:]

In [56]:
original_X_train = original_X_train.values 
original_X_val = original_X_val.values
original_X_test = original_X_test.values

original_y_train = original_y_train.values.reshape(-1, 1) 
original_y_val = original_y_val.values.reshape(-1, 1)
original_y_test = original_y_test.values.reshape(-1, 1)

original_scaler_x = StandardScaler()

original_X_train_scaled = original_scaler_x.fit_transform(original_X_train)
original_X_val_scaled = original_scaler_x.transform(original_X_val)
original_X_test_scaled = original_scaler_x.transform(original_X_test)

In [57]:
original_X_train_tensor = torch.tensor(original_X_train_scaled, dtype=torch.float32)
original_y_train_tensor = torch.tensor(original_y_train, dtype=torch.float32)

original_X_val_tensor = torch.tensor(original_X_val_scaled, dtype=torch.float32)
original_y_val_tensor = torch.tensor(original_y_val, dtype=torch.float32)

original_X_test_tensor = torch.tensor(original_X_test_scaled, dtype=torch.float32)
original_y_test_tensor = torch.tensor(original_y_test, dtype=torch.float32)

In [66]:
model = BPNeuralNetwork()

def negative_correlation_loss(output, target):

    output = output - torch.mean(output) 
    target = target - torch.mean(target)
    numerator = torch.sum(output * target) 
    denominator = torch.sqrt(torch.sum(output ** 2)) * torch.sqrt(torch.sum(target ** 2))  
    correlation = numerator / (denominator + 1e-8)  
    return -correlation 

criterion = negative_correlation_loss
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

patience = 10000  # Early stopping patience
best_val_loss = float('inf')
counter = 0
num_epochs = 1000

for epoch in range(num_epochs):
    # Training
    model.train()
    outputs = model(original_X_train_tensor).flatten()
    loss = criterion(outputs, original_y_train_tensor.flatten())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(original_X_val_tensor).flatten()
        val_loss = criterion(val_outputs, y_val_tensor.flatten())

    # Calculate correlation for training
    train_correlation = np.corrcoef(
        original_y_train_tensor.numpy().flatten(), outputs.detach().numpy().flatten()
    )[0, 1]

    # Calculate correlation for validation
    val_correlation = np.corrcoef(
        original_y_val_tensor.numpy().flatten(), val_outputs.numpy().flatten()
    )[0, 1]

    # Early Stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break

    # Print metrics every 10 epochs or at the last epoch
    if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
        print(
            f"Epoch [{epoch+1}/{num_epochs}], "
            f"Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
            f"Train Correlation: {train_correlation:.4f}, Val Correlation: {val_correlation:.4f}"
        )

model.eval()
with torch.no_grad():
    test_outputs = model(original_X_test_tensor).flatten()
    test_loss = criterion(test_outputs, original_y_test_tensor.flatten()).item()

    y_pred = test_outputs.numpy()
    y_test_array = original_y_test_tensor.numpy()

    test_correlation = np.corrcoef(y_test_array.flatten(), y_pred.flatten())[0, 1]

    print(f"Test Loss (Negative Correlation): {test_loss}")
    print(f"Correlation between y_test and y_pred: {test_correlation}")

Epoch [10/1000], Loss: -0.0337, Val Loss: 0.0420, Train Correlation: 0.0337, Val Correlation: -0.0420
Epoch [20/1000], Loss: -0.0531, Val Loss: 0.0338, Train Correlation: 0.0531, Val Correlation: -0.0338
Epoch [30/1000], Loss: -0.0723, Val Loss: 0.0257, Train Correlation: 0.0723, Val Correlation: -0.0257
Epoch [40/1000], Loss: -0.0933, Val Loss: 0.0178, Train Correlation: 0.0933, Val Correlation: -0.0178
Epoch [50/1000], Loss: -0.1185, Val Loss: 0.0111, Train Correlation: 0.1185, Val Correlation: -0.0111
Epoch [60/1000], Loss: -0.1504, Val Loss: 0.0010, Train Correlation: 0.1504, Val Correlation: -0.0010
Epoch [70/1000], Loss: -0.1830, Val Loss: -0.0102, Train Correlation: 0.1830, Val Correlation: 0.0102
Epoch [80/1000], Loss: -0.2112, Val Loss: -0.0135, Train Correlation: 0.2112, Val Correlation: 0.0135
Epoch [90/1000], Loss: -0.2374, Val Loss: -0.0116, Train Correlation: 0.2374, Val Correlation: 0.0116
Epoch [100/1000], Loss: -0.2611, Val Loss: -0.0069, Train Correlation: 0.2611, Val

Epoch [810/1000], Loss: -0.8885, Val Loss: 0.1202, Train Correlation: 0.8885, Val Correlation: -0.1202
Epoch [820/1000], Loss: -0.8902, Val Loss: 0.1207, Train Correlation: 0.8902, Val Correlation: -0.1207
Epoch [830/1000], Loss: -0.8920, Val Loss: 0.1209, Train Correlation: 0.8920, Val Correlation: -0.1209
Epoch [840/1000], Loss: -0.8938, Val Loss: 0.1208, Train Correlation: 0.8938, Val Correlation: -0.1208
Epoch [850/1000], Loss: -0.8956, Val Loss: 0.1209, Train Correlation: 0.8956, Val Correlation: -0.1209
Epoch [860/1000], Loss: -0.8972, Val Loss: 0.1208, Train Correlation: 0.8972, Val Correlation: -0.1208
Epoch [870/1000], Loss: -0.8988, Val Loss: 0.1209, Train Correlation: 0.8988, Val Correlation: -0.1209
Epoch [880/1000], Loss: -0.9002, Val Loss: 0.1210, Train Correlation: 0.9002, Val Correlation: -0.1210
Epoch [890/1000], Loss: -0.9018, Val Loss: 0.1212, Train Correlation: 0.9018, Val Correlation: -0.1212
Epoch [900/1000], Loss: -0.9032, Val Loss: 0.1211, Train Correlation: 0.9